In [1]:
import os
import gc
import time
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn import preprocessing
from contextlib import contextmanager

%matplotlib inline

In [2]:
@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

In [3]:
# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category=True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

In [4]:
# Preprocess bureau.csv and bureau_balance.csv
def bureau_and_balance(num_rows=None, nan_as_category=True):
    bureau = pd.read_csv('../data/raw/bureau.csv', nrows=num_rows)
    bb = pd.read_csv('../data/raw/bureau_balance.csv', nrows=num_rows)
    bb, bb_cat = one_hot_encoder(bb, nan_as_category)
    bureau, bureau_cat = one_hot_encoder(bureau, nan_as_category)
    
    # Bureau balance: Perform aggregations and merge with bureau.csv
    # -- MONTHS_BALANCE_MIN, MONTHS_BALANCE_MAX, MONTHS_BALANCE_SIZE group by SK_ID_BUREAU
    bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}
    # -- STATUS_X_MEAN group by SK_ID_BUREAU
    for col in bb_cat:
        bb_aggregations[col] = ['mean']
    bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
    bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])
    bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')
    bureau.drop(['SK_ID_BUREAU'], axis=1, inplace=True)
    del bb, bb_agg
    gc.collect()
    
    # Bureau and bureau_balance numeric features
#     num_aggregations = {
#         'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
#         'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
#         'DAYS_CREDIT_UPDATE': ['mean'],
#         'CREDIT_DAY_OVERDUE': ['max', 'mean'],
#         'AMT_CREDIT_MAX_OVERDUE': ['mean'],
#         'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
#         'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
#         'AMT_CREDIT_SUM_OVERDUE': ['mean'],
#         'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
#         'AMT_ANNUITY': ['max', 'mean'],
#         'CNT_CREDIT_PROLONG': ['sum'],
#         'MONTHS_BALANCE_MIN': ['min'],
#         'MONTHS_BALANCE_MAX': ['max'],
#         'MONTHS_BALANCE_SIZE': ['mean', 'sum']
#     }
    num_aggregations = {
        'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean', 'var'],
        'DAYS_CREDIT_UPDATE': ['min', 'max', 'mean', 'var'],
        'CREDIT_DAY_OVERDUE': ['max', 'min', 'mean', 'var'],
        'AMT_CREDIT_MAX_OVERDUE': ['min', 'max', 'mean', 'var'],
        'AMT_CREDIT_SUM': ['min', 'max', 'mean', 'sum', 'var'],
        'AMT_CREDIT_SUM_DEBT': ['min', 'max', 'mean', 'sum', 'var'],
        'AMT_CREDIT_SUM_OVERDUE': ['min', 'max', 'mean', 'var'],
        'AMT_CREDIT_SUM_LIMIT': ['min', 'max', 'mean', 'sum', 'var'],
        'AMT_ANNUITY': ['min', 'max', 'mean', 'var'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'MONTHS_BALANCE_MIN': ['min'],
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['mean', 'sum', 'var']
    }
    # Bureau and bureau_balance categorical features
    cat_aggregations = {}
    for cat in bureau_cat: cat_aggregations[cat] = ['mean']
    for cat in bb_cat: cat_aggregations[cat + "_MEAN"] = ['mean']
    
    bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])
    
    # Bureau: Active credits - using only numerical aggregations
    active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
    active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
    active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
    active_agg['ACTIVE_DEBT_CREDIT_RATIO'] = active_agg['ACTIVE_AMT_CREDIT_SUM_DEBT_SUM']/active_agg['ACTIVE_AMT_CREDIT_SUM_SUM']
    bureau_agg = bureau_agg.join(active_agg, how='left', on='SK_ID_CURR')
    del active, active_agg
    gc.collect()
    
    # Bureau: Closed credits - using only numerical aggregations
    closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
    closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
    closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
    closed_agg['CLOSED_DEBT_CREDIT_RATIO'] = closed_agg['CLOSED_AMT_CREDIT_SUM_DEBT_SUM']/closed_agg['CLOSED_AMT_CREDIT_SUM_SUM']

    bureau_agg = bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR')
    del closed, closed_agg, bureau
    gc.collect()
    
    # Bureau: DEBT_CREDIT_RATIO - AMT_CREDIT_SUM_DEBT/AMT_CREDIT_SUM
    return bureau_agg

In [8]:
def days_diff_agg(bureau_agg):
    bureau = pd.read_csv('../data/raw/bureau.csv')
    # 每次借款的間隔天數（是很規律的，還是臨時借的？）
    B = bureau.copy()
    # Groupby each Customer and Sort values of DAYS_CREDIT in ascending order
    grp = B[['SK_ID_CURR', 'SK_ID_BUREAU', 'DAYS_CREDIT']].groupby(by = ['SK_ID_CURR'])
    grp1 = grp\
        .apply(lambda x: x.sort_values(['DAYS_CREDIT'], ascending = False))\
        .reset_index(drop=True)
    print("Grouping and Sorting done")

    # Calculate Difference between the number of Days 
    grp1['DAYS_CREDIT1'] = grp1['DAYS_CREDIT']*-1
    grp1['DAYS_DIFF'] = grp1\
        .groupby(by = ['SK_ID_CURR'])['DAYS_CREDIT1']\
        .diff()
    grp1['DAYS_DIFF'] = grp1['DAYS_DIFF'].fillna(0).astype('uint32')
    print("Difference days calculated")

    grp2 = grp1.reindex(columns=["SK_ID_BUREAU", "DAYS_DIFF"])
    B = B.merge(grp2, on=['SK_ID_BUREAU'], how='left')
    
    days_diff_agg = B[['SK_ID_CURR', 'DAYS_DIFF']]\
        .groupby('SK_ID_CURR')\
        .agg({'DAYS_DIFF': ['mean', 'var', 'max']})
    days_diff_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in days_diff_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(days_diff_agg, how='left', on='SK_ID_CURR')
    
    del B, grp, grp1, grp2, days_diff_agg
    gc.collect()

    return bureau_agg

In [6]:
def main(sample=False, filename='../data/preprocess/bureau_features.csv'):
    if os.path.exists(filename):
        return pd.read_csv(filename)

    num_rows = 10000 if sample else None
    with timer("Process bureau and bureau_balance"):
        bureau = bureau_and_balance(num_rows, True)
        bureau = days_diff_agg(bureau)
        print("Bureau df shape:", bureau.shape)
        bureau.to_csv(filename)
        return bureau

In [9]:
df = main()
# Bureau df shape: (305811, 121)
# Bureau df shape: (305811, 187)

Grouping and Sorting done
Difference days calculated
Bureau df shape: (305811, 187)
Process bureau and bureau_balance - done in 758s


In [ ]:
for col in df.columns:
    print(col)

In [ ]:
df.columns